In [ ]:
%%capture
!pip install pdfplumber

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
for dirname, _, filenames in os.walk(r'gdrive/My Drive/Colab Notebooks/research/data/bertforquestionanswering'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

gdrive/My Drive/Colab Notebooks/research/data/bertforquestionanswering/config.json
gdrive/My Drive/Colab Notebooks/research/data/bertforquestionanswering/pytorch_model.bin
gdrive/My Drive/Colab Notebooks/research/data/bertforquestionanswering/special_tokens_map.json
gdrive/My Drive/Colab Notebooks/research/data/bertforquestionanswering/vocab.txt
gdrive/My Drive/Colab Notebooks/research/data/bertforquestionanswering/tokenizer_config.json


In [ ]:
import pdfplumber
import glob

symptoms_txt = ''
knowledge_txt = ''

symptoms_dir_path = r'/content/gdrive/MyDrive/Colab Notebooks/research/data/Tea/all-about-tea/Disease Identification/*.*'
knowledge_dir_path = r'/content/gdrive/MyDrive/Colab Notebooks/research/data/Tea/all-about-tea/Tea Knowledge Base/*.*'

for file in glob.glob(symptoms_dir_path, recursive=True):
    print(file)
    with pdfplumber.open(file) as pdf:
        for pdf_page in pdf.pages:
            single_page_text = pdf_page.extract_text()
            symptoms_txt = symptoms_txt + '\n' + single_page_text
    pdf.close()

for file in glob.glob(knowledge_dir_path, recursive=True):
    print(file)
    with pdfplumber.open(file) as pdf:
        for pdf_page in pdf.pages:
            single_page_text = pdf_page.extract_text()
            knowledge_txt = knowledge_txt + '\n' + single_page_text
    pdf.close()

/content/gdrive/MyDrive/Colab Notebooks/research/data/Tea/all-about-tea/Disease Identification/Tea Leaf Disease and Symptoms.pdf
/content/gdrive/MyDrive/Colab Notebooks/research/data/Tea/all-about-tea/Tea Knowledge Base/Tea.pdf
/content/gdrive/MyDrive/Colab Notebooks/research/data/Tea/all-about-tea/Tea Knowledge Base/Tea - Wikipedia.pdf


In [ ]:
print(symptoms_txt)


Tea Leaf Rust (Hemileia vastatrix):
Symptoms: The upper leaf surface develops small, yellowish spots that gradually enlarge and turn rusty
brown. These spots correspond to raised, orange-brown pustules on the lower leaf surface.
Disease: Tea leaf rust is caused by a fungus that attacks the tea leaves, leading to reduced
photosynthesis and overall plant weakness. Severe infections can result in defoliation and yield loss.
Tea Mosquito Bug (Helopeltis spp.):
Symptoms: Affected tea leaves show small, round, yellow or brown lesions with a characteristic shot-
hole appearance. These lesions can lead to leaf discoloration and distortion.
Disease: Tea mosquito bugs are sap-sucking insects that feed on young tea shoots. Their feeding causes
damage to the leaf tissues, leading to reduced growth and quality of tea leaves.
Tea Red Spider Mite (Oligonychus coffeae):
Symptoms: Infested tea leaves exhibit a stippled appearance, with tiny yellow spots or flecks on the
upper leaf surface. As the infe

In [ ]:
print(knowledge_txt)


Tea Q&A
Tea is one of the most popular beverages enjoyed by people all over the world. Whether it's
a warm cup in the morning to start the day or a refreshing iced tea in the a?ernoon, tea has
become an integral part of many cultures and lifestyles. In this arBcle, we will explore the
fascinaBng world of tea, from its origins and health benefits to brewing techniques and
popular varieBes.
What is Tea?
Tea is a beverage made from the leaves of the Camellia sinensis plant, The leaves are
processed and brewed to create different types of tea, each with its own unique flavor and
characterisBcs, The main types of tea include green tea, black tea, oolong tea, white tea, and
herbal tea.
Health Benefits of Tea
Tea is not only a delighMul beverage but also offers numerous health benefits, One of the key
reasons for its popularity is its high anBoxidant content, which helps protect the body
against free radicals and oxidaBve stress, Regular tea consumpBon has been linked to a
strengthened immun

In [ ]:
%%capture
!pip install transformers

**Question Answering with a Fine-Tuned BERT**

Bidirectional Encoder Representations from Transformers

Instead of looking at words in isolation, BERT, a transformer based model attempts to use the context of words to get embeddings. BERT uses several concepts of deep learning come up with a model that looks at context in a bidirectional fashion, leveraging information from the entire sentences as a whole through self-attention.

Example: Suppose Question is "How I can cross a road when their is no zebra crossing." and this question very similar to query "How I can cross road on zebra crossing." and thier is a chance that above two method give wrong answers but could handle that question. And we don't remove stoping words in the document because if it remove word like in above example "no" meaning of whole sentance become change. Bert uses stoping words to make relations between words.

*What is Model Fine-Tuning?*

BERT (Bidirectional Encoder Representations from Transformers) is a big neural network architecture, with a huge number of parameters, that can range from 100 million to over 300 million. So, training a BERT model from scratch on a small dataset would result in overfitting.

So, it is better to use a pre-trained BERT model that was trained on a huge dataset, as a starting point. We can then further train the model on our relatively smaller dataset and this process is known as model fine-tuning.

In [ ]:
import torch
from transformers import BertForQuestionAnswering


model = BertForQuestionAnswering.from_pretrained(r'gdrive/My Drive/Colab Notebooks/research/data/bertforquestionanswering')


from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained(r'gdrive/My Drive/Colab Notebooks/research/data/bertforquestionanswering')

model.eval()

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), ep

## Inferencing

In [ ]:
def answer_question(question, answer_text):

    input_ids = tokenizer.encode(question, answer_text, max_length=512, truncation=True)

    print('Query has {:,} tokens.\n'.format(len(input_ids)))

    sep_index = input_ids.index(tokenizer.sep_token_id)

    num_seg_a = sep_index + 1

    num_seg_b = len(input_ids) - num_seg_a

    segment_ids = [0]*num_seg_a + [1]*num_seg_b

    assert len(segment_ids) == len(input_ids)

    start_scores, end_scores = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]), return_dict=False)

    all_tokens = tokenizer.convert_ids_to_tokens(input_ids)

    print(' '.join(all_tokens[torch.argmax(start_scores) : torch.argmax(end_scores)+1]))
    print(f'score: {torch.max(start_scores)}')
    score = float(torch.max(start_scores))
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    answer = tokens[answer_start]

    for i in range(answer_start + 1, answer_end + 1):

        if tokens[i][0:2] == ' ':
            answer += tokens[i][2:]

        else:
            answer += ' ' + tokens[i]
    return answer, score

In [ ]:
user_question = " I have observed a concerning disease on my farm that affects the upper leaf surface of the tea plants. It begins with the appearance of small, yellowish spots, which unfortunately gradually grow in size and transform into a distinct rusty brown color.what is the disease?"

In [ ]:
answer_question(user_question, symptoms_txt)

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Query has 512 tokens.

tea leaf rust
score: 4.301675796508789


('tea leaf rust', 4.301675796508789)

## Evaluating

In [ ]:
import string, re

def predict(context, query):
    inputs = tokenizer.encode_plus(query, context, return_tensors='pt')
    outputs = model(**inputs)
    answer_start = torch.argmax(outputs[0])
    answer_end = torch.argmax(outputs[1]) + 1
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(\
                                                                inputs['input_ids'][0][answer_start : answer_end]))
    return answer


'''
Removing articles and punctuation and standardizing whitespace
'''
def normalize_text(s):

    def remove_articles(text):
        regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
        return re.sub(regex, ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))


def compute_f1(prediction,truth):
    pred_tokens = normalize_text(prediction).split()
    truth_tokens = normalize_text(truth).split()
    # if either the prediction or the truth is no-answer, then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    common_tokens = set(pred_tokens) & set(truth_tokens)
    if len(common_tokens) == 0:
        return 0
    precision = len(common_tokens) / len(pred_tokens)
    recall = len(common_tokens) / len(truth_tokens)
    return 2 * precision * recall / (precision + recall)


def give_an_answer(context, query, answer):
    prediction = predict(context, query)
    em_score = compute_exact_match(prediction, answer)
    f1_score = compute_f1(prediction, answer)
    print (f'Question : {query}')
    print (f'Prediction : {prediction}')
    print (f'True Answer : {answer}')
    print (f'EM : {em_score}')
    print (f'F1 : {f1_score}')
    print ('\n')
    return f1_score

In [ ]:
%%time

context = """ Queen are a British rock band formed in London in 1970. Their classic line-up was
              Freddie Mercury (lead vocals, piano), Brian May (guitar, vocals), Roger Taylor
              (drums, vocals) and John Deacon (bass). Their earliest works were influenced by
              progressive rock, hard rock and heavy metal, but the band gradually ventured into
              more conventional and radio-friendly works by incorporating further styles, such as
              arena rock and pop rock. """

queries = ["When was Queen found?",
           "Who were the classic members of Queen band?",
           "What kind of band they are?"]

answers = ["1970",
           "Freddie Mercury, Brian May, Roger Taylor and John Deacon",
           "rock"]


for q,a in zip(queries, answers):
    give_an_answer(context, q, a)

Question : When was Queen found?
Prediction : 1970
True Answer : 1970
EM : 1
F1 : 1.0


Question : Who were the classic members of Queen band?
Prediction : freddie mercury ( lead vocals , piano ) , brian may ( guitar , vocals ) , roger taylor ( drums , vocals ) and john deacon ( bass )
True Answer : Freddie Mercury, Brian May, Roger Taylor and John Deacon
EM : 0
F1 : 0.6923076923076924


Question : What kind of band they are?
Prediction : british rock
True Answer : rock
EM : 0
F1 : 0.6666666666666666


CPU times: user 5.99 s, sys: 10.6 ms, total: 6 s
Wall time: 10.3 s


In [ ]:
%%time

context = """ The COVID-19 pandemic, also known as the coronavirus pandemic, is an ongoing pandemic
              of coronavirus disease 2019 (COVID-19) caused by severe acute respiratory syndrome
              coronavirus 2 (SARS-CoV-2). It was first identified in December 2019 in Wuhan, China.
              The World Health Organization declared the outbreak a Public Health Emergency of
              International Concern in January 2020 and a pandemic in March 2020. As of 6 February
              2021, more than 105 million cases have been confirmed, with more than 2.3 million deaths
              attributed to COVID-19. Symptoms of COVID-19 are highly variable, ranging from none to
              severe illness. The virus spreads mainly through the air when people are near each
              other.[b] It leaves an infected person as they breathe, cough, sneeze, or speak and
              enters another person via their mouth, nose, or eyes. It may also spread via
              contaminated surfaces. People remain infectious for up to two weeks, and can spread
              the virus even if they do not show symptoms.[9]"""

queries = ["What is COVID-19?",
           "What is caused by COVID-19?",
           "How many cases have been confirmed from COVID-19?",
           "How many deaths have been confirmed from COVID-19?",
           "How is COVID-19 spread?",
           "How long can an infected person remain infected?",
           "Can a infected person spread the virus even if they don't have symptoms?",
           "What do elephants eat?"]

answers = ["an ongoing pandemic of coronavirus disease 2019",
           "severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2)",
           "more than 105 million cases",
           "more than 2.3 million deaths",
           "mainly through the air when people are near each other. It leaves an infected person as they breathe, cough, sneeze, or speak and enters another person via their mouth, nose, or eyes. It may also spread via contaminated surfaces.",
           "up to two weeks",
           "yes",
           ""]


for q,a in zip(queries,answers):
    give_an_answer(context,q,a)

Question : What is COVID-19?
Prediction : an ongoing pandemic of coronavirus disease 2019
True Answer : an ongoing pandemic of coronavirus disease 2019
EM : 1
F1 : 1.0


Question : What is caused by COVID-19?
Prediction : coronavirus disease 2019
True Answer : severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2)
EM : 0
F1 : 0.2


Question : How many cases have been confirmed from COVID-19?
Prediction : more than 105 million
True Answer : more than 105 million cases
EM : 0
F1 : 0.888888888888889


Question : How many deaths have been confirmed from COVID-19?
Prediction : 2 . 3 million
True Answer : more than 2.3 million deaths
EM : 0
F1 : 0.25


Question : How is COVID-19 spread?
Prediction : through the air when people are near each other
True Answer : mainly through the air when people are near each other. It leaves an infected person as they breathe, cough, sneeze, or speak and enters another person via their mouth, nose, or eyes. It may also spread via contaminated surfaces.

### Interface

In [ ]:
%%capture
!pip install gradio
!pip install --ignore-installed Pillow==9.0.0

In [ ]:
import gradio as gr

In [ ]:
def ans_genarator_disease(question):
  context = symptoms_txt
  ans, score = answer_question(question,context)
  return ans

examples = [
    ["I saw a concerning disease on my farm that affects the upper leaf surface of the tea plants. It begins with the appearance of small, yellowish spots, which unfortunately gradually grow in size and transform into a distinct rusty brown color. What is the disease?"],
    ["I observe certain symptoms on my tea leaves. The affected tea leaves display small, round lesions that appear either yellow or brown in color. These lesions have a distinct shot-hole appearance, giving the leaves a unique pattern. Over time, these lesions can cause the affected leaves to undergo discoloration and distortion. It is important for me, as a farmer, to closely monitor these symptoms and take appropriate measures to address the issue in order to maintain the health and productivity of my tea plants. What is the disease?"],
    ["There are infected tea leaves show small, circular, whitish spots on the upper leaf surface. These spots may have a yellow halo and gradually enlarge, leading to blighting. What is the disease?"],
    ["there are green, brown or orange cushion-like blotches on the leaf surface, what is the disease?"]
]

demo = gr.Interface(
    fn=ans_genarator_disease,
    inputs=gr.inputs.Textbox(lines=5, label="Describe your disease (Symptoms)"),
    outputs=gr.outputs.Textbox(label="Disease"),
    examples=examples,
    title='TeaHealthAI : Intelligent Disease Identification',
)

demo.launch(share=True)

/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:30: UserWarning: `optional` parameter is deprecated, and it has no effect
  super().__init__(
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:30: UserWarning: `numeric` parameter is deprecated, and it has no effect
  super().__init__(
/usr/local/lib/python3.10/dist-packages/gradio/outputs.py:22: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://791b531d4c523815e5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
import gradio as gr

def ans_genarator_knowledge(question):
  context = knowledge_txt
  ans, score = answer_question(question,context)
  return ans

examples = [
    ["What is Tea?"],
    ["What are the Health Benefits of Tea?"],
    ["Is tea healthier than coffee?"],
]

demo = gr.Interface(
    fn=ans_genarator_knowledge,
    inputs=gr.inputs.Textbox(lines=5, label="Ask anything about Tea"),
    outputs=gr.outputs.Textbox(label="Generated Text"),
    examples=examples,
    title='TeaHealthAI : Tea Domain Knowledge System',
)

demo.launch(share=True)

/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:30: UserWarning: `optional` parameter is deprecated, and it has no effect
  super().__init__(
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:30: UserWarning: `numeric` parameter is deprecated, and it has no effect
  super().__init__(
/usr/local/lib/python3.10/dist-packages/gradio/outputs.py:22: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://67ba6c3557e37f94be.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
